## Investigating two stochastic approaches to playing tic-tac-toe:
### 1. Naive stochastician: credit assignment is based on batch values
### 2. Clever stochastician: credit assignment is based on adversarial selection
### 3. Comparison of naive vs clever stochasticians

#### Note 1: One of the things we'll investigate is the existence of a first-mover advantage i.e. the first mover can afford to be careless about their first move whereas the second player can't.

#### Note 2: Some experiments will be duplicated so the notebook is self-contained. 

In [1]:
## load everything:

import numpy as np
from naive_stochastician import naive_stochastician
from clever_stochastician import clever_stochastician

### there are four possible permutations where player A moves first
### and so we'll use a 2-d boolean vector(ex. [0,1]) to denote the
### combination of interest:
## 1. naive(player A) vs naive(player B)
## 2. clever(player A) vs clever(player B)
## 3. naive(player A) vs clever(player B)
## 4. clever(player A) vs naive(player B)

def game_simulation(player_combo,num_games,random_start,depth,gamma):

    outcomes = np.zeros(num_games)
    
    initial_conditions = []
    
    for i in range(num_games):
        
        game = 1.0
        
        Z = np.zeros((3,3))
        X, O = np.random.choice(np.arange(9),2,replace=False)
        Z[int(X/3)][X % 3] = 1.0
        
        if random_start == 1.0:
            ## the second player plays randomly:
            Z[int(O/3)][O % 3] = -1.0
            
        else:
            ## the second player doesn't play randomly:
            if player_combo[1] == 1.0:
                P2 = clever_stochastician(-1.0*Z,depth,gamma)
                Z += -1.0*P2.move()
            else:   
                P2 = naive_stochastician(-1.0*Z,depth,gamma)
                Z += -1.0*P2.move()
        
        initial_conditions.append(np.copy(Z))
    
        while game == 1.0:
            ## player A move:
            if player_combo[0] == 1.0:
                P1 = clever_stochastician(Z,depth,gamma)
                Z += P1.move()
            else:  
                P1 = naive_stochastician(Z,depth,gamma)
                Z += P1.move()
            
            if P1.score(Z)[1] != 0.0:
                outcomes[i] = P1.score(Z)[1]
                
                game = 0.0
                
                break
            
            ## player B move:
            if player_combo[1] == 1.0:
                P2 = clever_stochastician(-1.0*Z,depth,gamma)
                Z += -1.0*P2.move()
            else:   
                P2 = naive_stochastician(-1.0*Z,depth,gamma)
                Z += -1.0*P2.move()
            
    return initial_conditions, outcomes

## 1. Analysis of first mover advantage for naive stochastician:
### 1. First move of player B is random.
### 2. First move of player B is non-random.

In [2]:
player_combo = np.array([0.0,0.0])
num_games = 100
random_start = 1.0
depth = 5
gamma = 0.9

_, outcomes = game_simulation(player_combo,num_games,random_start,depth,gamma)

In [3]:
np.mean(outcomes == 1.0), np.mean(outcomes == 0.5),np.mean(outcomes == -1.0) ## win, draw, loss percentages

(0.69, 0.28, 0.03)

In [7]:
0.69/0.03

23.0

## So there appears to be a clear first mover advantage. Now, what if the second move is non-random?

In [4]:
player_combo = np.array([0.0,0.0])
num_games = 100
random_start = 0.0
depth = 5
gamma = 0.9

_, outcomes = game_simulation(player_combo,num_games,random_start,depth,gamma)

In [6]:
np.mean(outcomes == 1.0), np.mean(outcomes == 0.5),np.mean(outcomes == -1.0) ## win, draw, loss percentages

(0.57, 0.43, 0.0)

### Interestingly, non-random play appears to decrease the probability of losses at the expense of win percentages. In some sense the outcomes are more predictable. 

## 2. Analysis of first mover advantage for clever stochastician:
### 1. First move of player B is random.
### 2. First move of player B is non-random.

In [8]:
player_combo = np.array([1.0,1.0])
num_games = 100
random_start = 1.0
depth = 5
gamma = 0.9

_, outcomes = game_simulation(player_combo,num_games,random_start,depth,gamma)

In [9]:
np.mean(outcomes == 1.0), np.mean(outcomes == 0.5),np.mean(outcomes == -1.0) ## win, draw, loss percentages

(0.57, 0.39, 0.04)

In [10]:
np.mean(outcomes == 1.0)/np.mean(outcomes == -1.0)

14.249999999999998

In [11]:
player_combo = np.array([1.0,1.0])
num_games = 100
random_start = 0.0
depth = 5
gamma = 0.9

_, outcomes = game_simulation(player_combo,num_games,random_start,depth,gamma)

In [13]:
np.mean(outcomes == 1.0), np.mean(outcomes == 0.5),np.mean(outcomes == -1.0) ## win, draw, loss percentages

(0.39, 0.61, 0.0)

## 3. Naive stochastician vs Clever stochastician where naive moves first:
### 1. First move of player B is random.
### 2. First move of player B is non-random.

In [14]:
player_combo = np.array([0.0,1.0])
num_games = 100
random_start = 1.0
depth = 5
gamma = 0.9

_, outcomes = game_simulation(player_combo,num_games,random_start,depth,gamma)

In [15]:
np.mean(outcomes == 1.0), np.mean(outcomes == 0.5),np.mean(outcomes == -1.0) ## win, draw, loss percentages

(0.67, 0.21, 0.12)

In [16]:
np.mean(outcomes == 1.0)/np.mean(outcomes == -1.0)

5.583333333333334

### This is a pretty good win percentage. 

In [17]:
player_combo = np.array([0.0,1.0])
num_games = 100
random_start = 0.0
depth = 5
gamma = 0.9

_, outcomes = game_simulation(player_combo,num_games,random_start,depth,gamma)

In [18]:
np.mean(outcomes == 1.0), np.mean(outcomes == 0.5),np.mean(outcomes == -1.0) ## win, draw, loss percentages

(0.71, 0.0, 0.29)

In [19]:
np.mean(outcomes == 1.0)/np.mean(outcomes == -1.0)

2.4482758620689657

### The naive stochastician appears to lack control of the board. 

## 4. Clever stochastician vs Naive stochastician where clever moves first:
### 1. First move of player B is random.
### 2. First move of player B is non-random.

In [20]:
player_combo = np.array([1.0,0.0])
num_games = 100
random_start = 1.0
depth = 5
gamma = 0.9

_, outcomes = game_simulation(player_combo,num_games,random_start,depth,gamma)

In [21]:
np.mean(outcomes == 1.0), np.mean(outcomes == 0.5),np.mean(outcomes == -1.0) ## win, draw, loss percentages

(0.89, 0.06, 0.05)

In [22]:
np.mean(outcomes == 1.0)/np.mean(outcomes == -1.0)

17.8

### Now, that's a big win-loss ratio. 

In [26]:
player_combo = np.array([1.0,0.0])
num_games = 100
random_start = 0.0
depth = 5
gamma = 0.9

_, outcomes = game_simulation(player_combo,num_games,random_start,depth,gamma)

In [27]:
np.mean(outcomes == 1.0), np.mean(outcomes == 0.5),np.mean(outcomes == -1.0) ## win, draw, loss percentages

(0.67, 0.11, 0.22)

In [28]:
np.mean(outcomes == 1.0)/np.mean(outcomes == -1.0)

3.0454545454545454

### Clever clearly has an advantage but also loses a surprising number of times. Right now I would rank the algorithms in the following order: naive stochastician, simple player, clever stochastician. 